In [16]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

from paths import *
from scripts.manipulation import remove_acentos
from scripts.manipulation import normalize_cols
from scripts import scrap_data
from scripts import manipulation
from scripts import io
from scripts import vis_graphs

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import yaml

import geopandas as gpd

# Taubate

- Nao temos acesso aos dados por municipio dos seguintes indicadores: 
 - Ocupação leitos UTI COVID
 - Leitos COVID/100 mil hab
 - Internações

In [6]:
sp_full.head()

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,obitos_novos,obitos_pc,obitos_mm7d,letalidade,nome_ra,cod_ra,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
0,Adamantina,3500105,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,000000000",RA de Presidente Prudente,9,Marília,5,33894,7398,41199,0,8.0,"-21,6820","-51,0737",9
1,Adolfo,3500204,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,000000000",RA de São José do Rio Preto,14,São José do Rio Preto,15,3447,761,21106,0,8.0,"-21,2325","-49,6451",9
2,Aguaí,3500303,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,000000000",RA de Campinas,4,São João da Boa Vista,14,35608,5245,47455,0,8.0,"-22,0572","-46,9735",9
3,Águas da Prata,3500402,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,000000000",RA de Campinas,4,São João da Boa Vista,14,7797,1729,14267,0,8.0,"-21,9319","-46,7176",9
4,Águas de Lindóia,3500501,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,000000000",RA de Campinas,4,Campinas,3,18374,3275,6013,0,8.0,"-22,4733","-46,6314",9


In [7]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])




In [69]:
cols = ['datahora','nome_munic','codigo_ibge','casos','casos_novos','obitos','obitos_novos','nome_drs']

mask = sp_full['nome_drs'].isin(['Taubaté'])
vale = sp_full[mask][cols]


rename_cols = {
    "casos":"casos_tbt",
    "casos_novos":"casos_novos_tbt",
    "obitos":"obitos_tbt",
    "obitos_novos":"obitos_novos_tbt",
}
vale_ = vale.groupby(by=['nome_drs','datahora'], as_index=False).sum()

vale_['nome_munic'] = 'Vale'
vale_['codigo_ibge'] = 0


vale = pd.concat([vale,vale_], axis=0)

vale_ = vale_.rename(columns = rename_cols).drop(columns=['nome_munic','codigo_ibge'])
vale = vale.merge(vale_, on=['nome_drs','datahora'])

In [71]:
cols = ['casos','obitos','casos_tbt','obitos_tbt']


dd_final_all = pd.DataFrame()

for municipio in vale['nome_munic'].unique():
    tbt = vale[vale['nome_munic']==municipio]

    for col in cols:
        

        tbt[f'{col}_shift_7'] = tbt[f'{col}'].shift(7)
        tbt[f'{col}_shift_14'] = tbt[f'{col}'].shift(14)


        tbt[f'{col}_7d'] = tbt[f'{col}'] - tbt[f'{col}_shift_7']
        tbt[f'{col}_14d'] = tbt[f'{col}'] - tbt[f'{col}_shift_14']


        tbt[f'{col}_shift_14'] = tbt[f'{col}_7d'].shift(7)

        tbt[f'{col}_var'] = tbt[f'{col}_7d']/tbt[f'{col}_shift_14']
        
        tbt = tbt.drop(columns = [f'{col}_shift_14',f'{col}_shift_7',])



#     dd = tbt[['datahora','nome_munic',f'{col}',f'{col}_7',f'{col}_14d',f'{col}_var']]

    dd_final_all = pd.concat([dd_final_all,tbt], axis=0)


    


vale_final = dd_final_all.copy()

In [72]:
vale_final['casos_tbt'] = 100 * vale_final['casos'] / vale_final['casos_tbt'] 
vale_final['casos_novos_tbt'] = 100 * vale_final['casos_novos'] / vale_final['casos_novos_tbt'] 
vale_final['obitos_tbt'] = 100 * vale_final['obitos'] / vale_final['obitos_tbt'] 
vale_final['obitos_novos_tbt'] = 100 * vale_final['obitos_novos'] / vale_final['obitos_novos_tbt'] 
vale_final = vale_final.fillna(0)

In [73]:
vale_final = vale_final.sort_values(by=['datahora','nome_munic'])
vale_final['codigo_ibge'] = vale_final['codigo_ibge'].astype(int)
vale_final['datahora'] = pd.to_datetime(vale_final['datahora'])

In [74]:
vale_final.tail()

,datahora,nome_munic,codigo_ibge,casos,casos_novos,obitos,obitos_novos,nome_drs,casos_tbt,casos_novos_tbt,obitos_tbt,obitos_novos_tbt,casos_7d,casos_14d,casos_var,obitos_7d,obitos_14d,obitos_var,casos_tbt_7d,casos_tbt_14d,casos_tbt_var,obitos_tbt_7d,obitos_tbt_14d,obitos_tbt_var
6156,2020-07-31,São Sebastião,3550704,859,11,14,0,Taubaté,5.073833,1.76,2.651515,0.0,217.0,229.0,18.083333,3.0,7.0,0.75,3731.0,6291.0,1.457422,84.0,168.0,1.0
6158,2020-07-31,Taubaté,3554102,1508,64,55,1,Taubaté,8.907265,10.24,10.416667,10.0,534.0,701.0,3.197605,8.0,24.0,0.50,3731.0,6291.0,1.457422,84.0,168.0,1.0
6159,2020-07-31,Tremembé,3554805,101,4,0,0,Taubaté,0.596574,0.64,0.000000,0.0,22.0,39.0,1.294118,0.0,0.0,0.00,3731.0,6291.0,1.457422,84.0,168.0,1.0
6160,2020-07-31,Ubatuba,3555406,498,8,20,0,Taubaté,2.941524,1.28,3.787879,0.0,125.0,227.0,1.225490,2.0,7.0,0.40,3731.0,6291.0,1.457422,84.0,168.0,1.0
6161,2020-07-31,Vale,0,16930,625,528,10,Taubaté,100.000000,100.00,100.000000,100.0,3731.0,6291.0,1.457422,84.0,168.0,1.00,3731.0,6291.0,1.457422,84.0,168.0,1.0


### RT

In [26]:
#load br cities
geo_sp = gpd.read_file('../../brasil_geodata/maps/sp_municipios.json')
# cols = ['geocodigo','nome_mesorregiao','geometry']
cols = ['geocodigo','nome_mesorregiao','nome_municipio']
geo_sp = geo_sp[cols]

In [27]:
#load state data
url  = 'http://datasource.coronacidades.org/'
data = 'br/states/rt'

ds = pd.read_csv(f'{url}{data}')

ds['last_updated'] = pd.to_datetime(ds['last_updated'])

sp = ds[ds['state']=='SP']
sp = sp.rename(columns={'state':'city_id'})
sp['nome_mesorregiao'] = 'SP'
sp['nome_municipio']   = 'Estado de SP'

rj = ds[ds['state']=='RJ']
rj = rj.rename(columns={'state':'city_id'})
rj['nome_mesorregiao'] = 'RJ'
rj['nome_municipio']   = 'Estado do RJ'

mg = ds[ds['state']=='MG']
mg = mg.rename(columns={'state':'city_id'})
mg['nome_mesorregiao'] = 'MG'
mg['nome_municipio']   = 'Estado de MG'

am = ds[ds['state']=='AM']
am = am.rename(columns={'state':'city_id'})
am['nome_mesorregiao'] = 'AM'
am['nome_municipio']   = 'Estado de AM'

In [28]:
#load cities data
data = 'br/cities/rt'

df = pd.read_csv(f'{url}{data}')

df['last_updated'] = pd.to_datetime(df['last_updated'])


In [94]:
dd = df.merge(geo_sp, how='right', left_on = 'city_id', right_on = 'geocodigo')
dd.head()


regiao = 'Vale do Paraíba Paulista'
mask = (dd['nome_mesorregiao']==regiao)
vale = dd[mask]
vale = vale[vale['city_id'].notnull()]

mask = (dd['nome_municipio']=='São Paulo')
sp_city = dd[mask]


vale = pd.concat([vale,sp_city], 0)
vale = pd.concat([vale,sp], 0)

vale['geocodigo'] = vale['geocodigo'].fillna(111).astype(int)

In [95]:
vale = vale.rename(columns={'geocodigo':'codigo_ibge','last_updated':'datahora'})
vale['datahora'] = pd.to_datetime(vale['datahora'])

In [96]:
mask = vale['datahora']== max(vale['datahora'])
vale_rt_last = vale[mask]
vale_rt_last = vale_rt_last.drop(columns=['datahora'])
vale_rt_last = vale_rt_last.rename(columns={'nome_municipio':'municipio'})

In [83]:
mask = vale_final['nome_munic']=='Vale'
just_vale = vale_final[mask]
just_vale = just_vale.drop(columns=['nome_munic'])

In [85]:
#upload to drive
from scripts import io
io.to_gbq(vale_final,'vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(just_vale,'just_vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(vale,'rt_vale_covid','covid','gabinete-sv', if_exists='replace')


1it [00:08,  8.03s/it]
1it [00:05,  5.12s/it]
1it [00:05,  5.62s/it]


In [98]:
io.to_gbq(vale_rt_last,'vale_rt_last','covid','gabinete-sv', if_exists='replace')

1it [00:08,  8.39s/it]


## Plano SP

In [71]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])

cols = ['datahora','nome_munic','casos','obitos','nome_drs']
sp = sp_full[cols][sp_full['nome_drs'].notnull()]

sp = sp.groupby(by=['nome_drs','datahora'], as_index=False).sum().sort_values(by=['nome_drs','datahora'])

estado = sp.groupby(by=['datahora'], as_index=False).sum().sort_values(by=['datahora'])
estado['nome_drs'] = 'Estado de São Paulo'
estado = estado[sp.columns]

sp = pd.concat([sp,estado],axis=0)

In [72]:
sp['casos'] = sp['casos'].astype(int)
sp['obitos'] = sp['obitos'].astype(int)

In [73]:
sp['nome_drs'].unique()

array(['Araraquara', 'Araçatuba', 'Baixada Santista', 'Barretos', 'Bauru',
       'Campinas', 'Franca', 'Grande São Paulo', 'Marília',
       'Município de São Paulo', 'Piracicaba', 'Presidente Prudente',
       'Registro', 'Ribeirão Preto', 'Sorocaba', 'São José do Rio Preto',
       'São João da Boa Vista', 'Taubaté', 'Estado de São Paulo'],
      dtype=object)

In [74]:
dd_final_all = pd.DataFrame()

for drs in sp['nome_drs'].unique():

    tbt = sp[sp['nome_drs']==drs]

    tbt['casos_shift'] = tbt['casos'].shift(7)
    tbt['obitos_shift'] = tbt['obitos'].shift(7)
    tbt['obitos_shift_14'] = tbt['obitos'].shift(14)


    tbt['casos_7'] = tbt['casos'] - tbt['casos_shift']
    tbt['obitos_7'] = tbt['obitos'] - tbt['obitos_shift']
    tbt['obitos_14d'] = tbt['obitos'] - tbt['obitos_shift_14']


    tbt['casos_7_shift'] = tbt['casos_7'].shift(7)
    tbt['obitos_7_shift'] = tbt['obitos_7'].shift(7)

    tbt['casos_var'] = tbt['casos_7']/tbt['casos_7_shift']
    tbt['obitos_var'] = tbt['obitos_7']/tbt['obitos_7_shift']


        
        
    dd = tbt[['datahora','nome_drs','casos_var','obitos_var','obitos_14d']]

    dd_final_all = pd.concat([dd_final_all,dd], axis=0)


    


dd_final = dd_final_all.copy()

## Internacoes

In [75]:
sp_cv = pd.read_csv('https://github.com/seade-R/dados-covid-sp/raw/master/data/plano_sp_leitos_internacoes.csv', sep=';')
sp_cv['internacoes_var'] = sp_cv['internacoes_7d']/sp_cv['internacoes_7d_l']
sp_cv['internacoes_14d'] = sp_cv['internacoes_7d'].astype(int) + sp_cv['internacoes_7d_l'].astype(int)
cols = ['datahora','nome_drs','leitos_pc','internacoes_var','internacoes_14d','pop']

uti_all = sp_cv[cols]

uti_all['nome_drs'] = uti_all['nome_drs'].apply(lambda x: x[7:] if x[:3]=='DRS' else x).str.strip()



df_final = uti_all[cols].copy()

In [76]:
sp_cv.head()

,datahora,nome_drs,total_covid_uti_mm7d,pop,leitos_pc,internacoes_7d,internacoes_7d_l,internacoes_7v7,internacoes_var,internacoes_14d
0,2020-05-19,Município de São Paulo,"2868,00",11869660,"24,16",5049,4728,"6,79",1.07,9777
1,2020-05-19,Grande SP Leste,"329,14",2977781,"11,05",596,488,"22,13",1.22,1084
2,2020-05-19,Grande SP Norte,"58,14",603465,"9,63",120,110,"9,09",1.09,230
3,2020-05-19,Grande SP Oeste,"211,14",1858252,"11,36",544,434,"25,35",1.25,978
4,2020-05-19,Grande SP Sudeste,"536,57",2690590,"19,94",970,825,"17,58",1.18,1795


In [77]:
df_final.tail()

,datahora,nome_drs,leitos_pc,internacoes_var,internacoes_14d,pop
1697,2020-07-31,São João da Boa Vista,"13,58",0.89,368,809836
1698,2020-07-31,São José do Rio Preto,"20,24",1.14,1091,1570421
1699,2020-07-31,Sorocaba,"9,88",1.00,842,2461760
1700,2020-07-31,Taubaté,"17,43",1.09,1410,2489629
1701,2020-07-31,Estado de São Paulo,"21,05",0.99,25732,44639899


In [78]:
dd_final.tail()

,datahora,nome_drs,casos_var,obitos_var,obitos_14d
153,2020-07-27,Estado de São Paulo,1.70,1.01,"3,777.00"
154,2020-07-28,Estado de São Paulo,2.15,1.02,"3,739.00"
155,2020-07-29,Estado de São Paulo,1.62,0.98,"3,757.00"
156,2020-07-30,Estado de São Paulo,1.55,0.97,"3,670.00"
157,2020-07-31,Estado de São Paulo,1.43,0.97,"3,647.00"


## Ocupacao UTI

In [79]:
uti = pd.read_csv('../data/sp_gov/uti_ocupacao.csv', sep=';')



uti_final = uti.copy()




# mask =  uti_final['datahora'] == max(uti_final['datahora'])

# uti_final = uti_final[mask]

# uti_final['datahora'] = max(df_final['datahora'])

In [80]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='inner')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')
df['pop'] = df['pop'].astype(int)

df['internacoes_14d_pc'] = (df['internacoes_14d'] * 10**5 )/df['pop']
df['obitos_14d_pc'] = (df['obitos_14d']  * 10**5 )/df['pop']

## Add Fases

In [97]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='inner')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')

df['pop'] = df['pop'].astype(int)

df['internacoes_14d_pc'] = (df['internacoes_14d'] * 10**5 )/df['pop']
df['obitos_14d_pc'] = (df['obitos_14d']  * 10**5 )/df['pop']

df = df.round(2)

def uti_fase(x):
    if x>=80:
        fase=1
    elif (x <80) & (x>=75):
        fase=2
    else:
        fase = 4
    return fase

def leitos_fase(x):
    if x<=3:
        fase=1
    elif (x >3) & (x<=5):
        fase=2
    else:
        fase = 4
    return fase


O = df['uti_var'].apply(lambda x: uti_fase(x))

L = df['leitos_pc'].str.replace(',','.').astype(float).apply(lambda x: leitos_fase(x))


c1 = (O*4 + L)/5

df['Capacidade Hospitalar'] = c1
# df['Capacidade Hospitalar'] = c1.astype(int)



def casos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=3
    else:
        fase = 4
    return fase

def internacoes_fase(x,y):
    if x>=1.5:
        fase=1
    elif (x <1.5) & (x>=1):
        fase=2
    elif (x <1) & (y>=40):
        fase=3
    elif (x <1) & (y<40):
        fase=4
    return fase

def obitos_fase(x,y):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=2
    elif (x <1) & (y>=5):
        fase=3
    elif (x <1) & (y<5):
        fase=4
    return fase

NC = df['casos_var'].apply(lambda x: casos_fase(x))

NI = df.apply(lambda x: internacoes_fase(x['internacoes_var'],x['internacoes_14d_pc']), axis=1)

NO = df.apply(lambda x: obitos_fase(x['obitos_var'],x['obitos_14d_pc']), axis=1)

c2 = (NC + 3*NI + NO)/5

df['Evolução da Pandemia'] = c2

df['casos_var'] = df['casos_var'].round(2).astype(str)
df['obitos_var'] = df['obitos_var'].round(2).astype(str)
df['internacoes_var'] = df['internacoes_var'].round(2).astype(str)
df['uti_var'] = df['uti_var'].round(2).astype(str)

df['Classif. Final'] = df[['Capacidade Hospitalar','Evolução da Pandemia']].min(axis=1)

# df['Classif. Final'] = df['Classif. Final'].astype(int).astype(str)
# df['Evolução da Pandemia'] = df['Evolução da Pandemia'].astype(int).astype(str)
# df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].astype(int).astype(str)

df['Classif. Final'] = df['Classif. Final'].round(1).astype(str)
df['Evolução da Pandemia'] = df['Evolução da Pandemia'].round(1).astype(str)
df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].round(1).astype(str)

df['internacoes_14d_pc'] = df['internacoes_14d_pc'].round(1).astype(str)
df['obitos_14d_pc'] = df['obitos_14d_pc'].round(1).astype(str)


## Padronize 

In [98]:

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','internacoes_14d_pc','obitos_var','obitos_14d_pc','Classif. Final']

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','obitos_var','Classif. Final']



rename_drs = {
    'Araraquara':'DRS 03 - Araraquara',
    'Araçatuba':'DRS 02 - Araçatuba',
    'Baixada Santista':'DRS 04 - Baixada Santista',
    'Barretos':'DRS 05 - Barretos',
    'Bauru':'DRS 06 - Bauru',
    'Campinas':'DRS 07 - Campinas',
    'Estado de São Paulo':'Estado de São Paulo',
    'Franca':'DRS 08 - Franca',
    'Marília':'DRS 09 - Marília',
    'Município de São Paulo':'DRS 01 - Município de São Paulo',
    'Piracicaba':'DRS 10 - Piracicaba',
    'Presidente Prudente':'DRS 11 - Presidente Prudente',
    'Registro':'DRS 12 - Registro',
    'Ribeirão Preto':'DRS 13 - Ribeirão Preto',
    'Sorocaba':'DRS 16 - Sorocaba',
    'São José do Rio Preto':'DRS 15 - São José do Rio Preto',
    'São João da Boa Vista':'DRS 14 - São João da Boa Vista',
    'Taubaté':'DRS 17 - Taubaté',
    'Estado de São Paulo':'00 - Estado de São Paulo'
}

df['nome_drs'] = df['nome_drs'].map(rename_drs)

ddf = df.copy()


In [100]:
cols_rename = {
    "Capacidade Hospitalar": "cap_hosp",
    "Evolução da Pandemia": "evolucao_pand",
    "Classif. Final": "classif_final",
    "nome_drs": "drs_name",
}


ddf = ddf.rename(columns=cols_rename)
mask = ddf['datahora']==max(ddf['datahora'])
ddf['last_update'] = max(ddf['datahora'])
ddf_last = ddf[mask].rename(columns={"drs_name":"DRS"})

#upload to drive
from scripts import io
io.to_gbq(ddf,'plano_sp','covid','gabinete-sv', if_exists='replace')
io.to_gbq(ddf_last,'plano_sp_last','covid','gabinete-sv', if_exists='replace')

1it [00:06,  6.75s/it]
1it [00:05,  5.73s/it]
